## Welcome to my final project!

### first importing all the necessary packages

In [89]:
import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
import json


from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

## Data Cleaning: Toronto

First I pulled the boroughs from wikipedia and put it into a dataframe. I proceeded to clean the boroughs by removing unassigned values

In [30]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)

boroughs = df_list[0]
boroughs = boroughs[boroughs['Borough'] != "Not assigned"]
boroughs.head()



,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The next step is to pull the geospatial data from a csv file, to mark the boundaries of the neighborhoods, and merge it with the boroughs dataframe

In [45]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
toronto = pd.merge(boroughs, coordinates, on = 'Postal Code')
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Data Cleaning: New York

For New York we repeat a similar process to early labs where we pull the data from a JSON file, and then generate a panda dataframe. We then iterate through all the data to store it in our new dataframe

In [38]:
with open('nyc_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
    
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

new_york = neighborhoods
new_york.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Data Visualization

we now have two similar data tables, one of New York neighborhood data and one of Toronto neighborhood data. Let's first look at the New York data.

In [42]:
latitude = new_york.iloc[4]['Latitude']
longitude = new_york.iloc[4]['Longitude']
# create map of Toronto using latitude and longitude values of downtown Toronto
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_york['Latitude'], new_york['Longitude'], new_york['Borough'], new_york['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    

map_newyork

Now let's visualize our toronto data, using the same process

In [47]:
latitude = toronto.iloc[4]['Latitude']
longitude = toronto.iloc[4]['Longitude']
# create map of Toronto using latitude and longitude values of downtown Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

map_toronto

## Using Foursquare to generate relevant information

First we use our unique client ID to generate a foursquare instance.

In [48]:
CLIENT_ID = 'K11VPNUOXKGYLELQWL135WET4ET1OFP2V3UJ5KAUPFD2MSAD' # your Foursquare ID
CLIENT_SECRET = 'ZVMHSAU0JHS0V014M4QI221ZZS3XFH2AXIO5BJMUFXZ552XT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: K11VPNUOXKGYLELQWL135WET4ET1OFP2V3UJ5KAUPFD2MSAD
CLIENT_SECRET:ZVMHSAU0JHS0V014M4QI221ZZS3XFH2AXIO5BJMUFXZ552XT


Next we create a function that builds Foursquare URL's to generate queires that report nearby venues, to all of our neighborhoods. We call the function on our New York as well as Toronto Datasets

In [52]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [53]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

nyc_venues = getNearbyVenues(names=new_york['Neighborhood'],
                                   latitudes=new_york['Latitude'],
                                   longitudes=new_york['Longitude']
                                  )

Now we want to remove the categories that are not coffee shops

In [58]:
toronto_coffee = toronto_venues[toronto_venues['Venue Category']=="Coffee Shop"]
nyc_coffee = nyc_venues[nyc_venues['Venue Category']=="Coffee Shop"]

toronto_coffee.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
19,"Regent Park, Harbourfront",43.654260,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop
21,"Regent Park, Harbourfront",43.654260,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
23,"Regent Park, Harbourfront",43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop


In [59]:
nyc_coffee.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
73,Kingsbridge,40.881687,-73.902818,Mon Amour Coffee & Wine,40.885009,-73.900332,Coffee Shop
135,Marble Hill,40.876551,-73.910660,Starbucks,40.877531,-73.905582,Coffee Shop
140,Marble Hill,40.876551,-73.910660,Starbucks,40.873755,-73.908613,Coffee Shop
152,Marble Hill,40.876551,-73.910660,Starbucks,40.873234,-73.908730,Coffee Shop
187,Norwood,40.877224,-73.879391,Nicky's Coffee Shop,40.874933,-73.879404,Coffee Shop


Now we visualize the location of these coffee shops in New York and Toronto

In [63]:
latitude = new_york.iloc[4]['Latitude']
longitude = new_york.iloc[4]['Longitude']
# create map of Toronto using latitude and longitude values of downtown Toronto
map_newyork_coffee = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(nyc_coffee['Neighborhood Latitude'], nyc_coffee['Neighborhood Longitude'], nyc_coffee['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork_coffee)  
    
map_newyork_coffee

In [64]:
latitude = toronto.iloc[4]['Latitude']
longitude = toronto.iloc[4]['Longitude']
# create map of Toronto using latitude and longitude values of downtown Toronto
map_toronto_coffee = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(toronto_coffee['Neighborhood Latitude'], toronto_coffee['Neighborhood Longitude'], toronto_coffee['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_coffee)  
    
map_toronto_coffee

# Now we want to pull in data on neighborhood demographics, to see information on the nieghborhoods that have more or less coffee shops

we first use another wikipedia page to pull demographic statistics on various neighborhoods, and then count the number of coffee shops in each neighborhood. Finally we merge the datatables to gether to get one master data table.

In [171]:
url = 'https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'
df_list = pd.read_html(url)

neighborhood_data = df_list[1]
neighborhood_data = neighborhood_data.append(df_list[2])
neighborhood_data = neighborhood_data.append(df_list[5])
neighborhood_data = neighborhood_data.rename(columns={"Name": "Neighbourhood"})
neighborhood_data = pd.merge(neighborhood_data, toronto, on = 'Neighbourhood')

neighborhood_data = neighborhood_data.rename(columns={ "Average Income":"Avg_income",
                                                     "Density (people/km2)":"Density",
                                                     "% Renters": "Renters"})
neighborhood_data.head()



,Neighbourhood,FM,Census Tracts,Population,Land area (km2),Density,% Change in Population since 2001,Avg_income,Transit Commuting %,Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map,Postal Code,Borough,Latitude,Longitude
0,Leaside,EY,"0195.00, 0196.00",13876.0,2.81,4938.0,3.0,82670.0,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,NaN,M4G,East York,43.709060,-79.363452
1,Thorncliffe Park,EY,"0194.01, 0194.02, 0194.03, 0194.04",17949.0,3.09,5809.0,9.1,25340.0,16.7,32.5,Urdu (21.5%),21.5% Urdu,NaN,M4H,East York,43.705369,-79.349372
2,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577.0,12.45,3580.0,4.6,25750.0,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,NaN,M1S,Scarborough,43.794200,-79.262029
3,Bayview Village,NY,"0305.01, 305.02",12280.0,4.14,2966.0,41.6,46752.0,14.4,15.6,Cantonese (8.4%),08.4% Cantonese,NaN,M2K,North York,43.786947,-79.385975
4,Church and Wellesley,OCoT,"0063.01, 0063.02",13397.0,0.55,24358.0,8.8,37653.0,25.1,57.0,Spanish (1.8%),01.8% Spanish,NaN,M4Y,Downtown Toronto,43.665860,-79.383160


In [172]:
total_coffee = pd.DataFrame(toronto_coffee['Neighborhood'].value_counts(), index = None)

total_coffee = total_coffee.reset_index()
total_coffee = total_coffee.rename(columns={"index": "Neighbourhood", "Neighborhood":"Coffee_Count"})

In [173]:
master_toronto = pd.merge(neighborhood_data, total_coffee, on = 'Neighbourhood', how='outer').fillna(0)

In [174]:
master_toronto.head()

,Neighbourhood,FM,Census Tracts,Population,Land area (km2),Density,% Change in Population since 2001,Avg_income,Transit Commuting %,Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map,Postal Code,Borough,Latitude,Longitude,Coffee_Count
0,Leaside,EY,"0195.00, 0196.00",13876.0,2.81,4938.0,3.0,82670.0,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,0.0,M4G,East York,43.709060,-79.363452,3.0
1,Thorncliffe Park,EY,"0194.01, 0194.02, 0194.03, 0194.04",17949.0,3.09,5809.0,9.1,25340.0,16.7,32.5,Urdu (21.5%),21.5% Urdu,0.0,M4H,East York,43.705369,-79.349372,1.0
2,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577.0,12.45,3580.0,4.6,25750.0,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,0.0,M1S,Scarborough,43.794200,-79.262029,0.0
3,Bayview Village,NY,"0305.01, 305.02",12280.0,4.14,2966.0,41.6,46752.0,14.4,15.6,Cantonese (8.4%),08.4% Cantonese,0.0,M2K,North York,43.786947,-79.385975,0.0
4,Church and Wellesley,OCoT,"0063.01, 0063.02",13397.0,0.55,24358.0,8.8,37653.0,25.1,57.0,Spanish (1.8%),01.8% Spanish,0.0,M4Y,Downtown Toronto,43.665860,-79.383160,7.0


We will now repeat the process for New York. Our dataset is a little different here, since wikipedia does not provide the same information. 

In [194]:
url = pd.read_csv("nyc_demographics")
url = url.drop(columns=['Geographic Area - Neighborhood Tabulation Area (NTA)* Code', 'Geographic Area - Borough', 'Geographic Area - 2010 Census FIPS County Code'])
nyc_data = url.rename(columns={"Geographic Area - Neighborhood Tabulation Area (NTA)* Name": "Neighborhood",
                              "Population":"Total Population 2010 Number"})
nyc_data = nyc_data.rename(columns={"Total Population 2010 Number":"Population"})
nyc_data = nyc_data.dropna()

total_coffee_nyc = pd.DataFrame(nyc_coffee['Neighborhood'].value_counts(), index = None)

total_coffee_nyc = total_coffee_nyc.reset_index()
total_coffee_nyc = total_coffee_nyc.rename(columns={"index": "Neighborhood", "Neighborhood":"Coffee_Count"})

In [195]:
master_nyc = pd.merge(nyc_data, total_coffee_nyc, on = 'Neighborhood', how='outer').fillna(0)

# Our Final Two DataFrames

In [196]:
master_nyc.head()

,Neighborhood,Total Population 2000 Number,Population,Total Population Change 2000-2010 Number,Total Population Change 2000-2010 Percent,Coffee_Count
0,Claremont-Bathgate,28149.0,31078.0,2929.0,10.4,0.0
1,Eastchester-Edenwald-Baychester,35422.0,34517.0,-905.0,-2.6,0.0
2,Bedford Park-Fordham North,55329.0,54415.0,-914.0,-1.7,0.0
3,Belmont,25967.0,27378.0,1411.0,5.4,2.0
4,Bronxdale,34309.0,35538.0,1229.0,3.6,0.0


In [197]:
master_toronto.head()

,Neighbourhood,FM,Census Tracts,Population,Land area (km2),Density,% Change in Population since 2001,Avg_income,Transit Commuting %,Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map,Postal Code,Borough,Latitude,Longitude,Coffee_Count
0,Leaside,EY,"0195.00, 0196.00",13876.0,2.81,4938.0,3.0,82670.0,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,0.0,M4G,East York,43.709060,-79.363452,3.0
1,Thorncliffe Park,EY,"0194.01, 0194.02, 0194.03, 0194.04",17949.0,3.09,5809.0,9.1,25340.0,16.7,32.5,Urdu (21.5%),21.5% Urdu,0.0,M4H,East York,43.705369,-79.349372,1.0
2,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577.0,12.45,3580.0,4.6,25750.0,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,0.0,M1S,Scarborough,43.794200,-79.262029,0.0
3,Bayview Village,NY,"0305.01, 305.02",12280.0,4.14,2966.0,41.6,46752.0,14.4,15.6,Cantonese (8.4%),08.4% Cantonese,0.0,M2K,North York,43.786947,-79.385975,0.0
4,Church and Wellesley,OCoT,"0063.01, 0063.02",13397.0,0.55,24358.0,8.8,37653.0,25.1,57.0,Spanish (1.8%),01.8% Spanish,0.0,M4Y,Downtown Toronto,43.665860,-79.383160,7.0


## Now let's explore some data analytics, first for Toronto

In [198]:
from statsmodels.formula.api import ols
model = ols(formula="Coffee_Count~Population + + I(Population**2) + Avg_income + Density + Renters ", data = master_toronto).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Coffee_Count   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     2.906
Date:                Fri, 28 Aug 2020   Prob (F-statistic):             0.0210
Time:                        22:54:38   Log-Likelihood:                -163.61
No. Observations:                  63   AIC:                             339.2
Df Residuals:                      57   BIC:                             352.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.0363      0

### Now let's do New York

In [199]:
from statsmodels.formula.api import ols
model = ols(formula="Coffee_Count~Population + + I(Population**2) ", data = master_nyc).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Coffee_Count   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     44.17
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           2.45e-17
Time:                        22:54:39   Log-Likelihood:                -525.40
No. Observations:                 275   AIC:                             1057.
Df Residuals:                     272   BIC:                             1068.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              2.3840      0